In [14]:
import xarray as xr
import iris
import numpy as np 
import pylab as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:

'''

Aim: To build a multifidelity ML model to predict high-resolution gridded
	surface temperature climate simulation out from contemporaneous 
	coarse-resolution climate simulation output

hi  = high fidelity data
lo  = low fidelity data

t2m  = temperature at 2 meters above the ground
u10, v10 = wind components (eastward, northward) at 10m above the ground

lat = latitude
lon = longitude

'''


### Regional gridded output from a Regional Climate Model (RCM)
hi_t2   = xr.open_dataset('data/hifid_t2m_monthly.nc')
hi_lats = hi_t2['latitude'].values
hi_lons = hi_t2['longitude'].values 

### Global gridded reanalysis (data assimilated) used to update/force the 
### lateral boundaries of the RCM
lo_t2  = xr.open_dataset('data/lofid_t2m_monthly.nc')
lo_u10 = xr.open_dataset('data/lofid_u10_monthly.nc')
lo_v10 = xr.open_dataset('data/lofid_v10_monthly.nc')

### work with numpy arrays
hi_t2_arr  = hi_t2['t2m'].values
lo_t2_arr  = lo_t2['t2m'].values 
lo_u10_arr = lo_u10['u10'].values
lo_v10_arr = lo_v10['v10'].values
time_dim   = lo_t2['time'].values 



In [5]:
def downscale_temperature(low_res_data, high_res_shape):
    # Reshape the low-resolution data
    low_res_data = low_res_data.reshape(low_res_data.shape[0], low_res_data.shape[1] * low_res_data.shape[2])
    
    # Define the model architecture
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(low_res_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(high_res_shape[1] * high_res_shape[2], activation='linear')) # output layer
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train the model
    model.fit(low_res_data, low_res_data, epochs=10)
    
    # Use the model to generate high-resolution temperature data
    high_res_data = model.predict(low_res_data)
    
    # Reshape the high-resolution data to the desired shape
    high_res_data = high_res_data.reshape(high_res_shape)
    
    return high_res_data

In [9]:
print(lo_t2_arr.shape)
x = lo_t2_arr.reshape(lo_t2_arr.shape[0],lo_t2_arr.shape[1]*lo_t2_arr.shape[2])
x.shape

(468, 54, 52)


(468, 2808)

In [17]:
np.random.randn(3,3,3)

array([[[ 1.37783439,  0.46208143, -0.46870421],
        [-0.38954503, -0.24992559,  0.94300567],
        [ 0.19851323,  0.82902809, -1.33960528]],

       [[-1.10016206, -0.16422231, -1.19520635],
        [-0.24979381,  0.78237043, -0.29071069],
        [-0.73365899,  0.08243652, -1.01530768]],

       [[ 0.2005561 , -0.09065383, -2.19386687],
        [ 0.92457954, -0.46785781, -0.1022121 ],
        [-0.64942146, -0.1043741 , -1.37738609]]])